In [1]:
from __future__ import division,print_function,absolute_import
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("Mnist",one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting Mnist\train-labels-idx1-ubyte.gz
Extracting Mnist\t10k-images-idx3-ubyte.gz
Extracting Mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate=0.001
num_steps=2000
batch_size=128

n_input=784
n_classes=10
dropout=0.25
#每次丢弃0.25的神经元，通过组合，增加识别精确度

In [3]:
def conv_net(x_dict,n_classes,dropout,reuse,is_training):
    with tf.variable_scope('ConvNet',reuse=reuse):
        x=x_dict['images']
        x=tf.reshape(x,shape=[-1,28,28,1])
        
        conv1=tf.layers.conv2d(x,32,5,activation=tf.nn.relu)
        #32 filters, ernel size=5
        #max pooling stride size=2,kernel size=5
        conv1=tf.layers.max_pooling2d(conv1,2,2)
        conv2=tf.layers.conv2d(conv1,64,3,activation=tf.nn.relu)
        conv2=tf.layers.max_pooling2d(conv2,2,2)
        
        #flatten the data to a 1-D vector for the fully connected layer
        fc1=tf.contrib.layers.flatten(conv2)
        #fully connected layer( in tf contrib folder for now)
        fc1=tf.layers.dense(fc1,1024)
        
        fc1=tf.layers.dropout(fc1,rate=dropout,training=is_training)
        
        out=tf.layers.dense(fc1,n_classes)
    return out
    

In [4]:
def model_fn(features,labels,mode):
    #构造网络，因为dropout在训练和预测的时候有不一样的表现，因此构建两个权值一样的运算图
    logits_train=conv_net(features,n_classes,dropout,reuse=False,is_training=True)
    logits_test=conv_net(features,n_classes,dropout,reuse=True,is_training=False)
    
    pred_classes=tf.argmax(logits_test,axis=1)
    pred_probas=tf.nn.softmax(logits_test)
    
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode,predictions=pred_classes)
    #define loss and optimizer:
    loss_op=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits_train,labels=tf.cast(labels,dtype=tf.int32)
    ))
    train_op=tf.train.AdamOptimizer(learning_rate=learning_rate).\
                minimize(loss_op,global_step=tf.train.get_global_step())
    acc_op=tf.metrics.accuracy(labels=labels,predictions=pred_classes)
    estim_specs=tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy':acc_op}
    )
    
    return estim_specs

In [5]:
model=tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\xenia\\AppData\\Local\\Temp\\tmpeg_t37zz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015E11ED6550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
input_fn=tf.estimator.inputs.numpy_input_fn(
x={'images':mnist.train.images},y=mnist.train.labels,
    batch_size=batch_size,num_epochs=None,shuffle=True)
model.train(input_fn,steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\xenia\AppData\Local\Temp\tmpeg_t37zz\model.ckpt.
INFO:tensorflow:loss = 2.3111591, step = 1
INFO:tensorflow:global_step/sec: 0.289803
INFO:tensorflow:loss = 0.15135555, step = 101 (345.093 sec)
INFO:tensorflow:Saving checkpoints for 165 into C:\Users\xenia\AppData\Local\Temp\tmpeg_t37zz\model.ckpt.
INFO:tensorflow:global_step/sec: 0.287769
INFO:tensorflow:loss = 0.15789896, step = 201 (347.503 sec)
INFO:tensorflow:global_step/sec: 0.291683
INFO:tensorflow:loss = 0.03879494, step = 301 (342.846 sec)
INFO:tensorflow:Saving checkpoints for 337 into C:\Users\xenia\AppData\Local\Temp\tmpeg_t37zz\model.ckpt.


In [ ]:
input_fn=tf.estimator.inputs.numpy_input_fn(
    x={'images':mnist.test.images},y=mnist.test.labels,
    batch_size=batch_size,shuffle=False)
model.evaluate(input_fn)

In [ ]:
n_images=4
test_images=mnist.test.images[:n_images]
input_fn=tf.estimator.inputs.numpy_input_fn(
    x={'images':test_images},shuffle=False)
preds=list(model.predict(input_fn))

for i in range(n_images):
    plt.imshow(np.reshape(test_images[i],[28,28]),comp='gray')
    plt.show()
    print("model prediction: ",preds[i])